In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Convolutional Neural Networks

In this assignment, we will learn about convolutional neural networks. We will create a CNN and learn to classify image data.

In this lecture, we will use the image data generator to classify our data. The data is loaded below:

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import (
    Activation,
    Dropout,
    Flatten,
    Dense,
    BatchNormalization,
)
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

<IPython.core.display.Javascript object>

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

<IPython.core.display.Javascript object>

In [4]:
tf.config.list_physical_devices("XLA_CPU")

[PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

<IPython.core.display.Javascript object>

In [5]:
physical_devices = tf.config.list_physical_devices("XLA_GPU")
tf.config.set_visible_devices(physical_devices[0], "XLA_GPU")

<IPython.core.display.Javascript object>

In [6]:
tf.config.PhysicalDevice(
  physical_devices[0], "XLA_GPU"
)


PhysicalDevice(name=PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), device_type='XLA_GPU')

<IPython.core.display.Javascript object>

In [7]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU')]

<IPython.core.display.Javascript object>

In [8]:
import os

tf.compat.v1.disable_eager_execution()

hello = tf.constant("Hello, TensorFlow!")
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

# sess = tf.compat.v1.Session()

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # You need to tell CUDA
# which GPU you'd like to use. if you have one GPU probably your GPU is '0'with tf.device('/gpu:0'):
with tf.device("/device:XLA_GPU:0"):
    a = tf.constant([1.0, 2.0, 3.0, 4.0], shape=[2, 2], name="a")
    b = tf.constant([4.0, 3.0, 2.0, 1.0], shape=[2, 2], name="b")
    c = tf.matmul(a, b)
    # with tf.compat.v1.Session() as sess:
    #     print(sess.run(c))

    sess = tf.compat.v1.Session(
        config=tf.compat.v1.ConfigProto(log_device_placement=True)
    )
    # Runs the op.
    print(sess.run(c))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

[[ 8.  5.]
 [20. 13.]]


<IPython.core.display.Javascript object>

In [9]:
train_data_dir = "data/dogs-vs-cats-processed/train/"
validation_data_dir = "data/dogs-vs-cats-processed/test/"

img_width, img_height = 150, 150
batch_size = 80

<IPython.core.display.Javascript object>

In [10]:
# This block of code is used to ensure the input shape is correct

if K.image_data_format() == "channels_first":
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

<IPython.core.display.Javascript object>

Define a train data generator with shear range of 0.3, zoom range of 0.1 and rescale to 1./255 (note that we must make 1 a float to produce a correct fraction). Use the ImageDataGenerator function.

In [11]:
# Answer below:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.3,
    zoom_range = 0.1,
    
)


<IPython.core.display.Javascript object>

Define a test data generator that only rescales to 1./255. Use the ImageDataGenerator function.

In [12]:
# Answer below:
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

<IPython.core.display.Javascript object>

The train generator and the test generator are defined below:

In [13]:
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   train_data_dir,
# #   validation_split=0.2,
# #   subset="training",
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)



<IPython.core.display.Javascript object>

In [14]:
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     validation_data_dir,
#     #   validation_split=0.2,
#     #       subset="validation",
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size,
# )

<IPython.core.display.Javascript object>

In [15]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    # shuffle=False,
    class_mode="binary",
)

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


<IPython.core.display.Javascript object>

We'll start with a simple model. In CNNs, we first convolve the to extract features and then we add the dense layers. 

Create a model with one layer of convolution of size 64, one layer of activation, one layer of max pooling with pool size (2,2) and then one flattening layer, one dense layer of unit size 64 with a ReLU activation and one dense output layer. The output layer should have a sigmoid activation.

In [16]:
# Answer below:
model = Sequential()

# model.add(
#     layers.experimental.preprocessing.Rescaling(
#         1.0 / 255,
#         input_shape=(img_height, img_width, 3),
#     )
# )
# # model.add(layers.experimental.preprocessing.RandomRotation(0.3))
# model.add(layers.experimental.preprocessing.RandomZoom(0.1))

# first hidden layer
model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# #second hidden layer
# model.add(Conv2D(32, (3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# #third hidden layer
# model.add(Conv2D(32, (3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())  # flatten rgb to 1D
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # randomly remove half of data to prevent overfitting

model.add(Dense(1))
model.add(Activation("sigmoid"))

<IPython.core.display.Javascript object>

Compile the model using RMSprop.

In [17]:
from tensorflow.compiler import xla

<IPython.core.display.Javascript object>

In [18]:
dir(xla)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'experimental',
 'xla_data_pb2']

<IPython.core.display.Javascript object>

In [19]:
# Answer below:

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer="rmsprop",
    metrics=["accuracy"],
)

<IPython.core.display.Javascript object>

Fit the model using a fit generator. Use 50 epochs, 25 training steps and 15 validation steps

In [20]:
nb_train_samples = 2000
nb_validation_samples = 802
epochs = 50
batch_size = 16

# num of steps usually num training samples//batchsize

<IPython.core.display.Javascript object>

In [21]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE

# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


<IPython.core.display.Javascript object>

In [22]:
physical_devices[0]

PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')

<IPython.core.display.Javascript object>

In [23]:
# tf.keras.backend.clear_session()
# tf.config.optimizer.set_jit(True)

<IPython.core.display.Javascript object>

In [24]:
# Answer below:

model.fit(
    train_generator, 
    steps_per_epoch=25, 
    epochs=50, 
    validation_data=validation_generator, 
    validation_steps = 15,
#     workers=-1,
#     use_multiprocessing=True
)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

Epoch 1/50
25/25 [==============================] - ETA: 0s - batch: 12.0000 - size: 80.0000 - loss: 0.6950 - accuracy: 0.5000WARNING:tensorflow:From C:\Users\b1t\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
25/25 [==============================] - 32s 1s/step - batch: 12.0000 - size: 80.0000 - loss: 0.6950 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.4991
Epoch 2/50
25/25 [==============================] - 30s 1s/step - batch: 12.0000 - size: 80.0000 - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.4973
Epoch 3/50
18/25 [====================>.........] - ETA: 7s - batch: 8.50

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

Create a new model by adding an additional group of convolution, activation and max pooling layers before the flatten layer. Make the convolution layer of unit size 32. Keep everything else the same.

In [ ]:
# Answer below:



Fit and compile the model in the same way you did with the previous model. How did the results improve?

In [ ]:
# Answer below:



Create a new model based on the model above. Add an additional dense layer of size 64 with a ReLU activation after the flatten layer.

In [ ]:
# Answer below:



Fit and compile in the same way as above. Describe the difference in performance and speed.

In [ ]:
# Answer below:



Fit and compile using the Adam optimizer. Describe the difference in performance between the Adam and RMSprop optimizers.

In [ ]:
# Answer below:

